In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, _tr, Timestamp, Context
from mand.core import ProfileMonitor, PrintMonitor
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import num, find
import datetime
from mand.lib.dbsetup import setUpDb

from mand.demos.trading import TradingBook, TradingPortfolio, MarketDataSource, MarketInterface

db = ObjectDb()
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x10efadad0>: 108, mem=True, ro=False: entities=9, map=2


In [2]:
def makeTree(names):
    ret = []
    for name in names:
        subs = [ TradingBook(name+str(i)) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        ret.append(p)
    return ret

with db:
    pAll = TradingPortfolio('TopOfTheHouse').write()
    subs = makeTree(['Eq-Prop', 'Eq-Inst', 'FX', 'Rates', 'Credit', 'Delta1', 'Loans', 'Commod', 'ETFs', 'Mtge'])
    pAll.setChildren(subs)
    
print pAll
print '# books:', len(pAll.books())
print '# children:', len(pAll.children())

# books: 100
# children: 10


In [3]:
with db:
    bExt  = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[0]
p2 = pAll.children()[1]

b1 = p1.children()[0]
b2 = p2.children()[0]

print bExt.meta.name()
print b1.meta.name()
print b2.meta.name()

Customer1
Eq-Prop0
Eq-Inst0


In [4]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

In [5]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

In [6]:
with db:
    TradeOpenEvent = _tr.TradeOpenEvent
    cf1 = _tr.ForwardCashflow()
    ins1 = _tr.Equity()
    ins2 = _tr.Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.69,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                         message='Sorry, the broker says you actually paid 69. signed: the middle office'
                        ).write(validTime=ev1.meta._timestamp.validTime)
    
    s1_ibm.update(last=177.68)
    s1_goog.update(last=856.13)
    
    ts6 = Timestamp()
    

In [7]:
pm = None

class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        def clocks(ts):
            def fn(node):
                obj = node.key[0]
                m = node.key[1].split(':')[-1]
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return dict( [ (node.tweakPoint, node) for node in nodes ] )
    
        allNodes = clocks(ts1)
    
        allNodes.update(clocks(ts2))
        nodes = allNodes.values() 
    
        # IRL, we'd sort these according to some business req...
        nodes = sorted(nodes, key = lambda node: node.key[0].meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n.tweakPoint] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        for n in nodes:
            tweaks[n.tweakPoint] = tsAmend
            name = n.key[0].meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n.tweakPoint] = ts2
            name = n.key[0].meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        displayHeader('%s' % title)
        displayListOfDicts(data, names=['Activity', 'PnL'] )
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 1196.00

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

In [8]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

In [9]:
db3 = db.copy()
p = db3._get(pAll.meta.path())
    
# Note misleading cum times below due to recursion...

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times


# PnL explain for TopOfTheHouse: 2216.00

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Portfolio:items|121|11,727,664|1,137|96,922|GetValue
|Portfolio:items|121|11,726,527|19,302|96,913|GetValue/Calc
|Report:data|1|6,123,947|14|6,123,947|GetValue
|Report:data|1|6,123,932|399|6,123,932|GetValue/Calc
|TradingContainer:NPV|11|6,116,075|125|556,006|GetValue
|TradingContainer:NPV|11|6,115,949|1,758|555,995|GetValue/Calc
|Workbook:items|1,104|4,831,296|10,852|4,376|GetValue
|Workbook:items|1,100|4,820,443|4,666,364|4,382|GetValue/Calc
|Root:Clocks|2|1,622,207|2,487|811,103|Context
|Portfolio:children|242|1,066,009|1,716|4,404|GetValue
|Portfolio:children|121|1,064,293|539,760|8,795|GetValue/Calc
|Root:End|1|568,400|47|568,400|Context
|Root:Activity Trading|1|536,880|45|536,880|Context
|Root:Activity Portfolio|1|521,213|44|521,213|Context
|Root:Activity MarketData|1|502,534|46|502,534|Context
|Root:Amend Trading|1|482,803|101|482,803|Context
|Root:Start|1|477,694|20|477,694|Context
|Root:Amend Portfolio|1|476,025|39|476,025|Context
|Root:Amend MarketData|1|467,263|39|467,263|Context
|Root:Start breaks|1|463,961|38|463,961|Context
|PortfolioUpdateEvent:children|121|443,712|2,450|3,667|GetValue
|TradingBook|102|407,885|407,885|3,998|Db.Get
|Equity:NPV|15|189,104|180|12,606|GetValue
|Equity:NPV|15|188,924|515|12,594|GetValue/Calc
|MarketInterface:spot|15|174,712|150|11,647|GetValue
|MarketInterface:spot|15|174,562|470|11,637|GetValue/Calc
|ExternalRefData:state|15|165,554|125|11,036|GetValue
|ExternalRefData:state|15|165,428|288|11,028|GetValue/Calc
|RefData:state|15|165,139|136|11,009|GetValue
|RefData:state|15|165,003|71,954|11,000|GetValue/Calc
|Clock:cutoffs|2,480|148,214|9,891|59|GetValue
|Clock:cutoffs|20|138,323|456|6,916|GetValue/Calc
|Clock:parent|20|105,036|389|5,251|GetValue
|Clock:parent|20|104,647|84,492|5,232|GetValue/Calc
|PortfolioUpdateEvent|12|55,443|55,443|4,620|Db.Get
|TradingBook:clock|2,200|44,170|21,613|20|GetValue
|RefDataUpdateEvent|9|42,420|42,420|4,713|Db.Get
|TradingPortfolio|10|41,571|41,571|4,157|Db.Get
|Clock|5|23,209|23,209|4,641|Db.Get
|TradingBook:clock|1,100|22,556|17,814|20|GetValue/Calc
|TradeOpenEvent|4|19,077|19,077|4,769|Db.Get
|TradeOpenEvent:ticket|66|15,723|586|238|GetValue
|TradingTicket|4|15,136|15,136|3,784|Db.Get
|Equity:refdata|15|13,695|165|913|GetValue
|Equity:refdata|15|13,530|485|902|GetValue/Calc
|ClockEvent:parent|8|10,607|81|1,325|GetValue
|Portfolio:books|231|9,271|2,941|40|GetValue
|ClockEvent|2|8,774|8,774|4,387|Db.Get
|MarketInterface:source|15|8,537|124|569|GetValue
|_WorkItemEvent:book2|66|8,519|324|129|GetValue
|MarketInterface:source|15|8,413|390|560|GetValue/Calc
|MarketInterface|2|8,200|8,200|4,100|Db.Get
|_WorkItemEvent:item|33|7,944|240|240|GetValue
|MarketDataSource|2|7,806|7,806|3,903|Db.Get
|Equity|2|7,703|7,703|3,851|Db.Get
|Portfolio:clock|242|7,697|2,285|31|GetValue
|Portfolio:books|121|6,330|4,215|52|GetValue/Calc
|Portfolio:clock|121|5,411|1,837|44|GetValue/Calc
|MarketDataSource:clock|30|4,897|285|163|GetValue
|MarketInterface|1|4,714|4,714|4,714|Db.Put
|MarketDataSource:clock|15|4,611|244|307|GetValue/Calc
|RootClock|1|4,534|4,534|4,534|Db.Get
|TradeOpenEvent:premium|33|3,796|248|115|GetValue
|ForwardCashflow|1|3,548|3,548|3,548|Db.Get
|Event:amends|230|1,590|1,590|6|GetValue
|Entity:clock|40|568|347|14|GetValue
|_WorkItemEvent:book1|66|336|336|5|GetValue
|TradeOpenEvent:quantity|66|298|298|4|GetValue
|RefDataUpdateEvent:data|58|290|290|5|GetValue
|Entity:clock|20|220|220|11|GetValue/Calc
|MarketInterface:sourceName|15|215|149|14|GetValue
|RootClock:cutoffs|52|188|188|3|GetValue
|TradeOpenEvent:action|33|188|188|5|GetValue
|TradeOpenEvent:unitPrice|33|186|186|5|GetValue
|ForwardCashflow:NPV|11|183|139|16|GetValue
|Equity:assetName|15|128|128|8|GetValue
|MarketInterface:sourceName|15|65|65|4|GetValue/Calc
|ForwardCashflow:NPV|11|43|43|3|GetValue/Calc
|Report:valuable|1|6|6|6|GetValue
|Report:ts1|1|3|3|3|GetValue
|Report:ts2|1|3|3|3|GetValue